In [0]:
import tensorflow as tf
import keras

model = keras.applications.vgg19.VGG19(include_top=True, weights=None, input_shape=(32,32,1), classes=10)

#set data augmentation
DATA_AUGMENTATION = True

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


In [0]:
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from keras.datasets import cifar10
import cv2
import numpy as np

def resize(mnist):
     #function resizes the dimensions of inputted dataset
     data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            data.append(resized_img)
     return data

#'cifar', 'fashion mnist', 'mnist'
DATASET_USED = 'mnist'


if DATASET_USED == "cifar":
  # Load the CIFAR10 data.
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
if DATASET_USED == "fashion mnist":
  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_train = np.array(resize(x_train))
  x_test = np.array(resize(x_test))

  x_train = x_train.reshape(x_train.shape[0], 32, 32, 1)
  x_test = x_test.reshape(x_test.shape[0], 32, 32, 1)

if DATASET_USED == "mnist":
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train = np.array(resize(x_train))
  x_test = np.array(resize(x_test))

  x_train = x_train.reshape(x_train.shape[0], 32, 32, 1)
  x_test = x_test.reshape(x_test.shape[0], 32, 32, 1)


In [0]:
from keras.preprocessing.image import ImageDataGenerator

if DATA_AUGMENTATION == True:
  datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

  # Compute quantities required for featurewise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(x_train)

In [0]:
import keras
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [0]:
import numpy as np
from keras.utils import to_categorical


y_train= to_categorical(y_train)
y_test = to_categorical(y_test)


In [12]:
batch_size = 126
epochs = 100

if DATA_AUGMENTATION == True:
  history = model.fit(datagen.flow(x_train, y_train,
            batch_size=batch_size),
            epochs=epochs, shuffle=True,
            validation_data=(x_test, y_test))
else:
  
  history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs, shuffle=True,
            validation_data=(x_test, y_test))



Epoch 1/100


KeyboardInterrupt: ignored

In [0]:
history_dict = history.history
print(history_dict.keys())

In [0]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss') # "bo" is for "blue dot"
plt.plot(epochs, val_loss, 'b', label='Validation loss') # b is for "solid blue line"
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [0]:
plt.clf() # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [0]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

nb_classes = 10

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.cuda()
        #classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu().detach()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu().detach()])

# Confusion matrix
lbllist= lbllist.numpy()
predlist= predlist.numpy()

print("confusion_matrix:")
conf_mat=confusion_matrix(lbllist, predlist)
print(conf_mat)
print('\n')

# Per-class accuracy
print("per-class accuracy:")
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)
print('\n')
print("prf:")

#PRF
prf = precision_recall_fscore_support(lbllist, predlist)
print("class precision:", prf[0], '\n')
print("class recall:", prf[1], '\n')
print("class f1-score:", prf[2], '\n')